In [1]:
import requests
import json
from langchain.utilities import BashProcess
import os
import time
from tenacity import retry, wait_exponential

In [2]:
#Note: The openai-python library support for Azure OpenAI is in preview.
import os
import openai
openai.api_type = "azure"
openai.api_base = "https://gcrgpt4aoai6c.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

# @retry(wait=wait_random_exponential(multiplier=1, max=60))
@retry(wait=wait_exponential(multiplier=1, min=20, max=60))
def call_llm(mode, c_code):
   prompt = [{"role":"system","content":"You are an assistant for automatic software verification. You take a C program with a loop and an assertion as input and you have two modes. In first mode, you can translate the verification of C program to SMT2 constraints with invariant as an unknown function such that these constraints can be consumed by Z3. They will include an initialization, one or more transition relations, and a property check. In second mode, you act as a translator from C programs to programs in Boogie programming language. Given an input program in C, annotated with assertions, your goal is to create a Boogie program annotated with loop invariants which prove that the assertions hold.\nChoose simple invariants when possible. Please avoid non-linear arithmetic, 2*x is OK but x*y is disallowed. . If mode is unspecified then output both the SMT formula and the annotated boogie program."},{"role":"user","content":"SMT\nint main() {\n  // variable declarations\n  int x;\n  int y;\n  // pre-conditions\n  (x = 1);\n  (y = 0);\n  // loop body\n  while ((y < 100000)) {\n    {\n    (x  = (x + y));\n    (y  = (y + 1));\n    }\n\n \n\n  }\n  // post-condition\nassert( (x >= y) );\n}"},{"role":"assistant","content":"(set-logic HORN)\n(declare-fun inv (Int Int) Bool)\n\n \n\n; Initial condition \n(assert (forall ((x Int)(y Int)) (=> ( and (= x 1) (= y 0)) (inv x y))))\n\n \n\n; Transition relation \n(assert (forall ((x Int)(y Int)(x1 Int)(y1 Int)) (=> (and (inv x y) (< y 100000) (= x1 (+ x y)) (= y1 (+ y 1)) ) (inv x1 y1))))\n\n \n\n; Property\n(assert (forall ((x Int)(y Int)) (=> (and (inv x y)(not (< y 100000))) (>= x y) )))\n\n \n\n(check-sat)\n(get-model)"},{"role":"user","content":"int main() {\n  // variable declarations\n  int x;\n  int y;\n  // pre-conditions\n  assume((x >= 0));\n  assume((x <= 2));\n  assume((y <= 2));\n  assume((y >= 0));\n  // loop body\n  while (unknown()) {\n    {\n    (x  = (x + 2));\n    (y  = (y + 2));\n    }\n\n \n\n  }\n  // post-condition\nif ( (y == 0) )\nassert( (x != 4) );\n\n \n\n}"},{"role":"assistant","content":"procedure main()\n{\n  var x: int;\n  var y: int;\n  var nondet: bool;\n  // pre-conditions\n  assume(0 <= x);\n  assume(x <= 2);\n  assume(y <= 2);\n  assume(y >= 0);\n  // loop body\n  havoc nondet;\n  while (nondet)\n  invariant x-y <= 2 && y - x <= 2;\n  {\n    havoc nondet;\n    x := x + 2;\n    y := y + 2;\n  }\n  // post-condition\n  if(y==0) {\n    assert(x != 4);\n  }\n}"}]
   prompt.append({"role":"user", "content":f"{mode}\n{c_code}"})

   response = openai.ChatCompletion.create(
   engine="gpt-4",
   messages = prompt,
   temperature=0.2,
   max_tokens=7000,
   top_p=0.95,
   frequency_penalty=0,
   presence_penalty=0,
   stop=None)
   return response.choices[0].message.content
   

In [3]:
# res = call_llm("SMT","""
# int main()
# {
#     int x = 0;
#     int y, z;

#     while(x < 5) {
#        x += 1;
#        if( z <= y) {
#           y = z;
#        }
#     }

#     assert (z >= y);
# }
# """)
# print(res)

In [4]:
# def convert_file(num):
#     with open(f"c_benchmark/{num}.c") as f:
#         print("-----------------------------------")
#         # Read the file f
#         c_code = f.read()
#         input_prompt = prompt(c_code)
#         print(input_prompt)
#         llm_response = call_llm(input_prompt)
#         print(llm_response)
#         output = parsePromptResponse(llm_response)[1]
#         # output = "(set-logic HORN)\n" + output
#         print(output)
#         print("-----------------------------------")

#         with open(f"z3/{num}.smt2", "w") as f:
#             f.write(output)


# convert_file(3)

In [5]:
# tot_calls = 1
# time_limit = 40

# calls = 0
# start_time = time.time()
# i = 1
# retry = 0
# fail_list = []
# while i <= 134:
#     calls = calls % tot_calls
#     if calls == 0: start_time = time.time()
#     with open(f"c_benchmark/{i}.c") as f:
#         try:
#             c_code = f.read()
#             output = call_llm("SMT", c_code)
#             calls += 1
#             print(i)

#             with open(f"z3/{i}.smt2", "w") as f:
#                 f.write(output)

#             if calls == tot_calls:
#                 elapsed_time = time.time() - start_time
#                 remaining_time = time_limit - elapsed_time
#                 if remaining_time > 0:
#                     print(f"Time Remaining: {remaining_time}")
#                     time.sleep(remaining_time)

#             i += 1
            
#         except Exception as e:
#             print(f"FAILED {i} - {str(e)}")
#             calls += 1
#             if calls == tot_calls:
#                 elapsed_time = time.time() - start_time
#                 remaining_time = time_limit - elapsed_time
#                 if remaining_time > 0:
#                     print(f"Time Remaining: {remaining_time}")
#                     time.sleep(remaining_time)

#             retry += 1
#             if retry == 10:
#                 fail_list.append(i)
#                 retry = 0
#                 continue
#     #   response_json = { "error": str(e) }
#     #   return json.dumps(response_json)


In [7]:
i = 52
retry = 0
fail_list = []
while i < 134:
    with open(f"c_benchmark/{i}.c") as f:
        try:
            c_code = f.read()
            output = call_llm("SMT", c_code)
            with open(f"z3/{i}.smt2", "w") as f:
                f.write(output)
            print(i)

            i += 1
            
        except Exception as e:
            print(f"FAILED {i} - {str(e)}")
            retry += 1
            if retry == 10:
                fail_list.append(i)
                retry = 0
                continue

52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133


FileNotFoundError: [Errno 2] No such file or directory: 'c_benchmark/134.c'

In [8]:
print(fail_list)

[]


In [9]:
# # Boogie
# for i in range(1, 134):
#     with open(f"c_benchmark/{i}.c") as f:
#         # print("-----------------------------------")
#         # Read the file f
#         c_code = f.read()
#         input_prompt = prompt2("Boogie", c_code)
#         # print(input_prompt)
#         llm_response = call_llm(input_prompt)
#         if llm_response is None:
#             print(f"FAILED {i}")
#             continue
#         output = parsePromptResponse(llm_response)[1]
        
#         # output = "(set-logic HORN)\n" + output
#         # print(output)
#         # print("-----------------------------------")
#         print(i)
#         # x = input()
#         # if x == "q": break

#         with open(f"boogie/{i}.bpl", "w") as f:
#             f.write(output)

i = 1
retry = 0
fail_list = []
while i < 134:
    with open(f"c_benchmark/{i}.c") as f:
        try:
            c_code = f.read()
            output = call_llm("Boogie", c_code)
            with open(f"boogie/{i}.bpl", "w") as f:
                f.write(output)
            print(i)

            i += 1
            
        except Exception as e:
            print(f"FAILED {i} - {str(e)}")
            retry += 1
            if retry == 10:
                fail_list.append(i)
                retry = 0
                continue

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
